In [1]:
import torch
import math

In [3]:
class legendrePolynomial3(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return 0.5*(5* input**3 - 3*input)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input **2 -1)

In [4]:
dtype = torch.float
device = torch.device("cuda:0")

In [5]:
x = torch.linspace(-math.pi, math.pi, 2000, dtype = torch.float,device = device)
y = torch.sin(x)

In [6]:
a = torch.full((),0.0, device = device, dtype=  dtype, requires_grad = True)
b = torch.full((),-1.0, device = device, dtype=  dtype, requires_grad = True)
c = torch.full((),0.0, device = device, dtype=  dtype, requires_grad = True)
d = torch.full((),0.3, device = device, dtype=  dtype, requires_grad = True)

In [ ]:
learning_rate = 1e-6
for i in range(2000):
    p3 = legendrePolynomial3.apply
    
    y_pred = a + b * p3(c+d*x)
    
    loss = (y_pred - y).pow(2).sum()
    if i%100 == 99:
        print(i, loss.item())
        
    loss.backward()
    
    
    with torch.no_grad():
        
        a-= learning_rate * a.grad
        b-= learning_rate * b.grad
        c-= learning_rate * c.grad
        d -= learning_rate * d.grad
        
        
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
        
print(f"Results y = {a.item()} + {b.item() * p3({c.item()} + {d.item()} x^3 ")